In [ ]:
import tensorflow as tf
import pandas as pd
df=pd.read_csv('GOOG_data.csv')

In [ ]:
df.head()
df.shape

In [ ]:
df1=df.reset_index()['close']

In [ ]:
df1.head()


In [ ]:
import matplotlib.pyplot as plt
plt.plot(df1)

In [ ]:
import numpy as np
df1

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
df1=scaler.fit_transform(np.array(df1).reshape(-1,1))

In [ ]:
df1

In [ ]:
training_size=int(len(df1)*0.80)
test_size=len(df1)-training_size
train_data,test_data=df1[0:training_size,:],df1[training_size:len(df1),:1]

In [ ]:
training_size,test_size

In [ ]:
import numpy
def create_dataset(dataset, time_step=1):
     dataX, dataY = [] ,[]
     for i in range(len(dataset)-time_step-1):
         a = (dataset[i:(i+time_step),0])
         dataX.append(a)
         dataY.append(dataset[i + time_step, 0])
     return numpy.array(dataX) ,  numpy.array(dataY)   

In [ ]:
time_step=100
x_train, y_train = create_dataset(train_data, time_step)
x_test, y_test = create_dataset(test_data, time_step)

In [ ]:
print(x_train.shape),print(y_train.shape)

In [ ]:
print(x_test.shape), print(y_test.shape)

In [ ]:
x_train = x_train.reshape(x_train.shape[0],x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0],x_test.shape[1], 1)

In [ ]:
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers.coremodels import Dense
#from tensorflow.keras.models import LSTM
#import tensorflow.keras as keras
#from tensorflow.keras.layers import dense
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import Input
from tensorflow.python.keras.layers.recurrent import LSTM

In [ ]:
model = keras.Sequential()
model.add(keras.layers.LSTM(100, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(keras.layers.LSTM(100, return_sequences=False))
model.add(keras.layers.Dense(25))
model.add(keras.layers.Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')
model.summary()

In [ ]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=50,batch_size=64,verbose=1)

In [ ]:
tf.__version__

In [ ]:
train_predict=model.predict(x_train)
test_predict=model.predict(x_test)

In [ ]:
train_predict=scaler.inverse_transform(train_predict)
test_predict=scaler.inverse_transform(test_predict)

In [ ]:
import math
from sklearn.metrics import mean_squared_error
math.sqrt(mean_squared_error(y_train,train_predict))


In [ ]:
math.sqrt(mean_squared_error(y_test,test_predict))


In [ ]:
look_back=100
trainPredictPlot = numpy.empty_like(df1)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict

testPredictPlot = numpy.empty_like(df1)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(df1)-1, :] = test_predict

plt.plot(scaler.inverse_transform(df1))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.legend(['Train', 'Valid', 'Predictions'])
plt.show()


